In [1]:
# knn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

housing_maindf = pd.read_csv('~/Documents/datacamp/WBS-School/wbs-housing-iter-0-2/iter-5/housing-classification-iter5.csv')

housing_df = housing_maindf.copy()




In [3]:
# housing_df.info()

In [4]:
# housing_df.isna().sum()

In [5]:
X = housing_df.copy()


In [6]:
# target
y = X.pop("Expensive")


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31416)



In [8]:
# helper
def to_df(arr,df):
    print(arr.shape)
    print(df.shape)
    return pd.DataFrame(arr, columns=df.columns, index=df.index)


In [9]:
from sklearn.impute import SimpleImputer

# selecting non-numerical columns
X_train_cat = X_train.select_dtypes(exclude="number")

# defining the imputer to use "N_A" as replacement value
cat_imputer = SimpleImputer(strategy="constant", 
                            fill_value="N_A")#.set_output(transform='pandas')

# fitting and transforming
X_cat_imputed = cat_imputer.fit_transform(X_train_cat)
X_cat_imputed = to_df(X_cat_imputed,X_train_cat)



(1168, 14)
(1168, 14)


In [10]:
# Selecting numerical columns
X_train_num = X_train.select_dtypes(include="number")

# Imputing the mean
num_imputer = SimpleImputer(strategy="mean")#.set_output(transform='pandas')

# Fitting and transforming
X_num_imputed = num_imputer.fit_transform(X_train_num)
X_num_imputed = to_df(X_num_imputed,X_train_num)
X_num_imputed.head()

(1168, 36)
(1168, 36)


,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSSubClass,...,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold
772,7819.0,94.00000,1029.0,3.0,1.0,0.0,2.0,144.0,0.0,80.0,...,1.0,6.0,1976.0,672.0,0.0,0.0,0.0,0.0,3.0,2010.0
157,12003.0,92.00000,774.0,4.0,1.0,0.0,3.0,0.0,0.0,60.0,...,1.0,8.0,2009.0,680.0,75.0,0.0,0.0,0.0,5.0,2010.0
360,7540.0,70.10352,888.0,2.0,1.0,0.0,2.0,0.0,192.0,85.0,...,1.0,5.0,1978.0,470.0,0.0,0.0,0.0,0.0,6.0,2007.0
744,5395.0,41.00000,1337.0,2.0,1.0,0.0,2.0,96.0,0.0,120.0,...,1.0,5.0,1993.0,462.0,0.0,70.0,168.0,0.0,10.0,2008.0
150,10356.0,120.00000,969.0,3.0,0.0,0.0,2.0,0.0,0.0,20.0,...,1.0,5.0,1975.0,440.0,0.0,0.0,0.0,0.0,1.0,2007.0


In [11]:
from sklearn.preprocessing import OneHotEncoder

# initialize
# my_onehot = OneHotEncoder(drop="first",sparse_output=False).set_output(transform='pandas')
my_onehot = OneHotEncoder()
# fit
my_onehot.fit(X_cat_imputed)

# transform
X_cat_imputed_mat = my_onehot.transform(X_cat_imputed)

In [12]:
# X_cat_imputed_onehot = to_df(X_cat_imputed_onehot)
import scipy.sparse
X_cat_imputed_onehot = pd.DataFrame.sparse.from_spmatrix(X_cat_imputed_mat)

In [13]:
X_imputed = pd.concat([X_cat_imputed_onehot, X_num_imputed], axis=1)

X_imputed.head(3)

,0,1,2,3,4,5,6,7,8,9,...,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,8.0,2003.0,548.0,61.0,0.0,0.0,0.0,2.0,2008.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,6.0,1976.0,460.0,0.0,0.0,0.0,0.0,5.0,2007.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,6.0,2001.0,608.0,42.0,0.0,0.0,0.0,9.0,2008.0


In [14]:
from sklearn.pipeline import make_pipeline

# select categorical and numerical column names
X_cat_columns = X.select_dtypes(exclude="number").copy().columns
X_num_columns = X.select_dtypes(include="number").copy().columns

# create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
 # create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    # OneHotEncoder(sparse_output=False)
    OneHotEncoder()
)

In [15]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_pipe", categoric_pipe, X_cat_columns),
    ]
)

In [37]:
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [38]:
from sklearn.model_selection import GridSearchCV

knclass = KNeighborsClassifier()
full_pipeline = make_pipeline(preprocessor, knclass)
k_range = list(range(1, 31))

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors":k_range
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=0,
                      scoring='accuracy'
                     )


In [19]:
# knnreg.get_params().keys()

In [ ]:
search.fit(X_train, y_train)

In [21]:
search.best_score_

0.9135174791827152

In [22]:
search.best_params_

{'columntransformer__num_pipe__simpleimputer__strategy': 'mean',
 'kneighborsclassifier__n_neighbors': 5}

In [23]:
search.cv_results_['mean_test_score']

array([0.87927442, 0.89725249, 0.91351381, 0.89895455, 0.91351748,
       0.90066028, 0.90923664, 0.89723781, 0.90665053, 0.89894721,
       0.90151865, 0.89552474, 0.90066395, 0.88954184, 0.89725249,
       0.88611936, 0.88954184, 0.88526833, 0.89125491, 0.88354426,
       0.88697773, 0.88098382, 0.88525733, 0.87926708, 0.88012545,
       0.87670298, 0.87841238, 0.87156377, 0.87670298, 0.87241847,
       0.87927442, 0.89725249, 0.91351381, 0.89895455, 0.91351748,
       0.90066028, 0.90923664, 0.89723781, 0.90665053, 0.89894721,
       0.90151865, 0.89552474, 0.90066395, 0.88954184, 0.89725249,
       0.88611936, 0.88954184, 0.88526833, 0.89125491, 0.88354426,
       0.88697773, 0.88098382, 0.88525733, 0.87926708, 0.88012545,
       0.87670298, 0.87841238, 0.87156377, 0.87670298, 0.87241847])

In [24]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5, shuffle=False).split(range(25))

In [41]:
# from sklearn.model_selection import cross_val_score

# knn = KNeighborsRegressor(n_neighbors=5)
# scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
# print(scores)
# print(scores.mean())


In [28]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1'...
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                   

In [29]:
y_train_pred = full_pipeline.predict(X_train)

accuracy_score(y_train, y_train_pred)

/Users/cytalladen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9366438356164384

In [31]:
full_pipeline.fit(X_test, y_test)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1'...
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                   

In [32]:
y_test_pred = full_pipeline.predict(X_test)

accuracy_score(y_test, y_test_pred)

/Users/cytalladen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9383561643835616